## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-31-04-06-50 +0000,nypost,More than 30 bodies recovered from Houston-are...,https://nypost.com/2025/12/30/us-news/more-tha...
1,2025-12-31-04-00-00 +0000,wsj,Ukraine Now Has Europe’s Biggest Military. Wha...,https://www.wsj.com/world/ukraine-military-siz...
2,2025-12-31-03-58-02 +0000,nyt,"Tatiana Schlossberg, Kennedy Daughter Who Wrot...",https://www.nytimes.com/2025/12/30/us/politics...
3,2025-12-31-03-56-00 +0000,wsj,Tech Startups Are Handing Out Free Nicotine Po...,https://www.wsj.com/tech/tech-startups-are-han...
4,2025-12-31-03-54-13 +0000,nyt,Eurostar Trains Face Day of Delays After Power...,https://www.nytimes.com/2025/12/30/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2474/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
89,trump,28
5,year,12
25,dies,11
10,ukraine,11
296,new,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...,80
67,2025-12-30-23-13-27 +0000,nypost,Trump admin freezes all childcare payments to ...,https://nypost.com/2025/12/30/us-news/trump-ad...,76
24,2025-12-31-02-28-03 +0000,startribune,Trump administration says it’s freezing child ...,https://www.startribune.com/trump-administrati...,69
58,2025-12-30-23-54-11 +0000,bbc,Trump administration says it's withholding chi...,https://www.bbc.com/news/articles/c75xnndvlyko...,66
262,2025-12-30-04-18-08 +0000,nypost,Trump admin pausing some federal funding to Mi...,https://nypost.com/2025/12/29/us-news/trump-ad...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,80,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...
72,61,2025-12-30-23-05-25 +0000,cbc,Israel will ban dozens of aid groups from Gaza...,https://www.cbc.ca/radio/asithappens/gaza-aid-...
257,41,2025-12-30-05-14-28 +0000,bbc,Bangladesh's first female prime minister Khale...,https://www.bbc.com/news/articles/cm2v256vdkyo...
157,37,2025-12-30-18-18-44 +0000,nypost,Zohran Mamdani names two controversial lawyers...,https://nypost.com/2025/12/30/us-news/zohran-m...
176,34,2025-12-30-16-55-56 +0000,nypost,"Meta snaps up AI startup Manus for $2B, drawin...",https://nypost.com/2025/12/30/business/meta-sn...
92,33,2025-12-30-22-13-33 +0000,nypost,Most Fed officials see more rate cuts ahead as...,https://nypost.com/2025/12/30/business/most-fe...
15,31,2025-12-31-02-59-35 +0000,nypost,Hair-raising video captures Air Force sergeant...,https://nypost.com/2025/12/30/us-news/air-forc...
173,31,2025-12-30-17-06-18 +0000,nypost,"China unleashes hail of live fire in intense, ...",https://nypost.com/2025/12/30/world-news/china...
35,30,2025-12-31-01-49-23 +0000,nypost,Billionaire Bill Ackman warns ‘no one would st...,https://nypost.com/2025/12/30/us-news/californ...
125,29,2025-12-30-20-17-27 +0000,cbc,"Tatiana Schlossberg, granddaughter of the late...",https://www.cbc.ca/news/world/tatiana-schlossb...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
